In [ ]:
import os
import shutil
import csv
import argparse
import pickle
import numpy as np
import pandas as pd
import random
import pprint
# connect to sigopt api, below is my account token
from sigopt import Connection
conn = Connection(client_token="YOUR_CLIENT_TOKEN")

# I have removed randomization in modify_data for some need
from modify_data import CIFData
import run_cgcnn as rc

## create an experiment or retrieve an experiment

In [ ]:
experiment = conn.experiments().create(
    name='foo', #fill in experiment name 
    parameters=[
        dict(name='epochs',type='int',bounds=dict(min=50,max=500)),
        dict(name="log_learning_rate",bounds=dict(min=-18.4207, max=-1),type="double"),
        dict(name='n_conv', type='int', bounds=dict(min=1, max=6)),
        dict(name='n_h', type='int', bounds=dict(min=1, max=3)),
        dict(name='h_fea_len', type='int', bounds=dict(min=32, max=256)),
        dict(name='atom_fea_len', type='int', bounds=dict(min=4, max=256)),
        dict(name='batch_size',type='int',bounds=dict(min=32, max=512)),
        dict(name="optimizer",categorical_values=[dict(name="Adam"),dict(name="sgd")],type="categorical"),
        dict(name='neighbors',type='int',bounds=dict(min=6, max=12)),
        dict(name='bondstep',type='categorical',categorical_values=[{'name':'0.05'},{'name':'0.1'},{'name':'0.2'}])
    ],
    observation_budget = 100
)
print("Create experiment:" + experiment.id)

In [ ]:
# retrieve the experiment
experiment = conn.experiments(id=).fetch() 

## defind train/valid/test size as 6:2:2

In [ ]:
root_dir = '/root_dir/' 

files = os.listdir(root_dir)
data_size = len([f for f in files if f.endswith('.cif')])
train_size, val_size = round(data_size * 0.6),  round(data_size*0.2)
test_size = data_size - train_size - val_size
print(data_size, train_size, val_size, test_size)

In [ ]:
# load data
dataset = CIFData(root_dir) 

# define train model funtion
run_model = rc.run_model

# start optimizing
if conn.experiments(experiment.id).suggestions().fetch().data != []:
    print('not first experiment')
    suggestion = conn.experiments(experiment.id).suggestions().fetch().data[0]
    assignments = suggestion.assignments 
    print(assignments)
    value = run_model(assignments, root_dir, dataset, train_size, val_size, test_size, evaluate=True)
    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value)
else:
    print('first experiment')
print(experiment.observation_budget)

 
for i in range(experiment.observation_budget):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    assignments = suggestion.assignments
    print('========================================')
    print('iteration %d: suggestion assignments'%i)
    print(assignments)
    
    value = run_model(assignments, root_dir, dataset, train_size, val_size, test_size, evaluate=True)
    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value)